## --- [Day 16: Ticket Translation](https://adventofcode.com/2020/day/16) ---

In [1]:
import csv
import numpy as np
import re
from typing import List

INPUT_RULES = 'input1_d16.txt' # This file contains all the rules for ticket fields
INPUT_TIX = 'input2_d16.txt'  # Your ticket on row 0, all other tickets follow
EX_RULES = 'input3_d16.txt' # Example rules
EX_TIX = 'input4_d16.txt'   # Example tix, yours on row 0


class TicketTranslator:
    def __init__(self, rules_file: str):
        self.all_valid = set()
        self.rules = {}
        self.load_rules_file(rules_file)
        self.valid_tix = []
        self.invalid_tix = []
        self.error_rate = 0
    
    def load_rules_file(self, rules_file: str):
        '''
        Reads and parses a file of rules in the format
            field name: low-high or low-high
        :param rules_file: filename containing 1+ rules
        '''
        expr = r"^(?P<field>[\w\s]+): (?P<lo1>\d+)-(?P<hi1>\d+) or (?P<lo2>\d+)-(?P<hi2>\d+)"
        with open(rules_file) as fh:
            for line in fh.readlines():
                r = re.match(expr, line).groupdict()
                # Create a set with both ranges in it
                range1 = range(int(r['lo1']), int(r['hi1'])+1)
                range2 = range(int(r['lo2']), int(r['hi2'])+1)
                self.rules[r['field']] = set(range1)
                self.rules[r['field']].update(range2)
                # Also add all of those values to the set of ALL valid values
                self.all_valid.update(self.rules[r['field']])
    
    def scan_ticket(self, ticket: List[int]):
        '''
        Scans a single ticket
        :param ticket: list of int values, fields unknown
        '''
        # A ticket is valid when all of its fields are within *some* valid range
        if self.all_valid.issuperset(ticket):
            self.valid_tix.append(ticket)
        else:
            self.invalid_tix.append(ticket)
            self.error_rate += sum(set(ticket).difference(self.all_valid))
        
    
    def load_tix_file(self, tix_file: str):
        '''
        Loads a file containing tickets and scans each one.
        Each ticket is a comma-delimited string of integers
        :param tix_file:
        '''
        with open(tix_file) as fh:
            reader = csv.reader(fh)
            for row in reader:
                self.scan_ticket([int(v) for v in row])
                
    def possible_fields(self):
        '''
        Compares the values of each unknown field across all *valid* tickets
        to determine which fields each one could be based on the rules defined.
        :return: a list where length = total # of fields, element i is set of possible fields for field i
        '''
        # Each column will get a set of field names such that possible_fields[i]
        # is the set of field names that could correspond to field i
        possible_fields = []
        
        for col in np.array(self.valid_tix).transpose():
            colset = set(col)
            possible = set()
            
            # Check the set of values for this column against all rules
            for field, rule_set in self.rules.items():
                if colset.issubset(rule_set):
                    possible.update([field])
            
            possible_fields.append(possible)
        
        return possible_fields
    
    def match_fields(self):
        '''
        Uses set logic to match up each field with exactly 1 descriptor
        :return: list of field names such that element i is the name of field i
        '''
        possible_fields = self.possible_fields()
        
        # As each field narrowed to exactly 1 possibility, add to solved
        solved = set()
        last_n_solved = -1
        
        while len(solved) < len(possible_fields) and len(solved) > last_n_solved:
            # solved should grow by at least 1 each iteration if there's a solution
            last_n_solved = len(solved)
            
            for i in range(len(possible_fields)):
                if len(possible_fields[i]) > 1:
                    # try to narrow this set down by removing fields matched elsewhere
                    possible_fields[i].difference_update(solved)
                if len(possible_fields[i]) == 1:
                    # once the set of possibilities is 1, this field is matched
                    solved.update(possible_fields[i])
        
        return [s.pop() for s in possible_fields]
            
    def decode_my_ticket(self):
        '''
        Combines "my" ticket (assumed to be the first valid ticket scanned) with the matched
        fields
        :return: dict of {field: value}
        '''
        ticket = {}
        fields = self.match_fields()
        
        for i in range(len(fields)):
            ticket[fields[i]] = self.valid_tix[0][i]
        
        return ticket      
        

In [2]:
# Test example
ex = TicketTranslator(EX_RULES)
ex.load_tix_file(EX_TIX)
assert len(ex.valid_tix) == 2
assert len(ex.invalid_tix) == 3
assert ex.error_rate == 71

In [3]:
# Part 1 solution
p1 = TicketTranslator(INPUT_RULES)
p1.load_tix_file(INPUT_TIX)
print('Ticket scanning error rate was', p1.error_rate)

Ticket scanning error rate was 20013


### Part 2

Now that you've identified which tickets contain invalid values, discard those tickets entirely. Use the remaining valid tickets to determine which field is which.

Using the valid ranges for each field, determine what order the fields appear on the tickets. The order is consistent between all tickets: if seat is the third field, it is the third field on every ticket, including your ticket.

Once you work out which field is which, look for the six fields on your ticket that start with the word departure. What do you get if you multiply those six values together?

In [4]:
# NumPy is a handy way to transpose an array without iterating
vt = np.array(ex.valid_tix)
print(vt)
for col in vt.transpose():
    print(col)

[[ 7  1 14]
 [ 7  3 47]]
[7 7]
[1 3]
[14 47]


In [5]:
# Test against part 2 example
ex2 = TicketTranslator('input5_d16.txt')
ex2.load_tix_file('input6_d16.txt')
assert len(ex2.rules) == 3
assert len(ex2.valid_tix) == 4
# The number of fields matched should be the same as the number of fields on "my" ticket
assert len(ex2.match_fields()) == len(ex2.valid_tix[0])

In [6]:
# Part 2 solution
my_data = p1.decode_my_ticket()

# find the keys that start with 'departure'
depart_keys = list(filter(lambda f: f[0:10] == 'departure ', my_data))

solution = 1
for key in depart_keys:
    solution *= my_data[key]
    
solution

5977293343129